In [5]:
import torch
from torch import Tensor
from torch.utils.data import DataLoader
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt
import h5py

from methods.base import BaseConfig
from methods.plotting import plot_perm, plot_press, plot_event_list, plot_events_projection

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu' # my gpu is to old for cuda
print(device)

train_h5_path = 'train.h5'
test_h5_path = 'test.h5'

cpu


### Data retrieving example

In [ ]:
jj = 142 

with h5py.File(train_h5_path, 'a') as f:
    ev = f['events'][jj]
    pp = f['perm'][jj]
    pore = f['pore'][jj]
    ev_d = f['ev_dens'][jj]

params = BaseConfig()
params.load() # loading params from params.yaml

print('Permeability map')
plot_perm(pp, loc=(10, 10, 10), params=params)

print('Pore pressure at the fimal step')
plot_press(pore, loc=(10, 10, 10), params=params)

print('Events')
plot_event_list(ev[0:900], params=params)

print('Seismic density at the fimal step')
plot_events_projection(ev_d, params=params)

print('Pore pressure at the fimal step at source location slice')
fig, ax = plt.subplots()
ims = ax.imshow(pore[:,:,10]) # pore press at source location slice
plt.colorbar(ims, ax=ax)

In [44]:
# Dataset Class
class EventsToPermDataset(torch.utils.data.Dataset):

    def __init__(self, h5_path, transforms):
        super().__init__()
        self.path = h5_path
        self.transforms = self.stack_transforms(transforms) # transforms func
        with h5py.File(h5_path, 'a') as f:
            nmodels = f['events'].shape[0]
        self.nmodels = nmodels

    def __getitem__(self, idx):
        with h5py.File(self.path, 'a') as f:
            event = f['events'][idx]
            perm = f['perm'][idx]

        return self.transforms(event, perm)

    def __len__(self):
        return self.nmodels
    
    def stack_transforms(self, func_list):
        def performer(*args):
            for f in func_list:
                args = f(*args)
            return args
        return performer
    
class Normalizer:
    ''' returns log10(perm)     '''
    def __call__(self, events, perms):
        log_perm = torch.log10(perms)
        return events, log_perm

class ToTensor:
    '''Transforms numpy to torch tensors'''
    def __call__(self, events, perms):
        # perms = torch.tensor(perms, dtype=torch.float)
        # events = torch.tensor(events, dtype=torch.float)
        perms = torch.Tensor(perms)
        events = torch.Tensor(events)
        return events, perms


In [43]:
transforms = [ToTensor(), Normalizer(),] # data transformations

dataset = EventsToPermDataset(h5_path = train_h5_path, transforms=transforms)

dataloader = DataLoader(dataset=dataset,
                      batch_size=10,
                      shuffle=False,
                      )

In [86]:
# some mock model just for testing

class MockFc(nn.Module):
    '''
    Fc layer
    '''
    def __init__(self, input_shape, target_shape, fc_drop=0.1):
        super().__init__()
        self.input_shape = input_shape
        self.target_shape = target_shape

        input_flat_len = torch.zeros(self.input_shape).flatten().shape[0]
        target_flat_len = torch.zeros(self.target_shape).flatten().shape[0]
        
        self.fc1 = nn.Linear(input_flat_len, input_flat_len)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(fc_drop)
        self.fc2 = nn.Linear(input_flat_len, target_flat_len)

    def forward(self, x):
        bs = x.shape[0] # batch size
        x = x.flatten(0)
        x = self.drop(self.relu(self.fc1(x)))
        x = self.fc2(x)
        x = x.view(bs, *self.target_shape)
        return x


class MockEvensToPerm(nn.Module):
    '''
    Model itself
    list of events -> permeability map
    '''
    def __init__(self, input_shape, target_shape):
        super().__init__()
        self.input_shape = input_shape
        self.target_shape = target_shape
        self.fc = MockFc(self.input_shape, self.target_shape)

    def forward(self, x):
        x = self.fc(x)
        return x

In [89]:
X, y = dataset[0]

model = MockEvensToPerm(X.shape, y.shape)
                        
y_hat = model(X.unsqueeze(0)).squeeze(0)
y_hat
    

tensor([[[ 3.7144, -1.3563,  0.5127,  ..., -0.9286,  4.4130,  2.1863],
         [ 1.1066, -0.6971,  2.4352,  ..., -1.5593,  0.5824, -0.8085],
         [ 0.4792, -1.2389,  2.6676,  ...,  1.3547,  0.9999, -0.4382],
         ...,
         [ 1.7099, -1.2539, -1.5202,  ...,  1.2491,  0.3756,  0.0615],
         [-3.4966, -1.1421, -4.5903,  ..., -1.9084,  2.0402,  0.8846],
         [ 1.5455,  1.3725, -3.6487,  ..., -2.3872,  2.8364, -0.0309]],

        [[-1.9420, -3.4582, -1.0865,  ..., -0.8014,  2.1682, -3.1231],
         [ 3.9426, -1.0389, -0.1007,  ...,  1.0171, -0.0357,  2.0183],
         [-0.5261, -2.4663,  2.3317,  ..., -0.5341,  3.2944, -2.4095],
         ...,
         [ 4.2126, -3.5338, -1.1002,  ...,  2.1183,  0.7094, -0.7294],
         [ 1.6446,  1.4953,  0.0085,  ...,  1.2181,  0.9027,  2.9585],
         [-0.0262, -0.4036, -0.8714,  ..., -1.4647,  0.0601,  1.5520]],

        [[ 2.4198, -0.2293,  0.3937,  ..., -3.8375, -0.5768, -1.3801],
         [-0.8696,  1.8189, -3.2175,  ..., -0

In [80]:
X.unsqueeze(0).shape

torch.Size([1, 1100, 5])